# Tutorial 2: Grid Queries & Spatial Search

**Level:** Basic  
**Time:** 20-25 minutes  
**Prerequisites:** Tutorial 1 (Token Operations)

## Overview

This tutorial covers spatial indexing and queries in NeuroGraph:
- Creating and managing grids
- Finding neighbors within radius
- Range queries (bounding box)
- Field influence calculations
- Density mapping

## What are Grids?

Grids provide **spatial indexing** for efficient token queries:
- **Cell-based partitioning**: O(1) cell lookup
- **Neighbor search**: Find tokens within radius
- **Range queries**: Get tokens in bounding box
- **Field calculations**: Compute spatial influence

## Setup

In [ ]:
import requests
import json
import numpy as np
import matplotlib.pyplot as plt
from pprint import pprint

# API configuration
BASE_URL = "http://localhost:8000/api/v1"
headers = {"Content-Type": "application/json"}

# Helper function
def show(response):
    print(f"Status: {response.status_code}")
    if response.status_code == 200:
        pprint(response.json())
    else:
        print(response.text)

## Step 1: Authentication

In [ ]:
# Login
login_data = {"username": "admin", "password": "admin"}
response = requests.post(f"{BASE_URL}/auth/login", json=login_data)
access_token = response.json()["access_token"]
headers["Authorization"] = f"Bearer {access_token}"
print("✓ Authenticated")

## Step 2: Create a Grid

Create a spatial index with defined cell size.

In [ ]:
# Create grid with 2.0 unit cells
grid_data = {
    "cell_size": 2.0,
    "dimensions": 8
}

response = requests.post(f"{BASE_URL}/grid", json=grid_data, headers=headers)
grid = response.json()
grid_id = grid["grid_id"]

print(f"✓ Grid created with ID: {grid_id}")
print(f"  Cell size: {grid['cell_size']}")
print(f"  Dimensions: {grid['dimensions']}")

## Step 3: Populate Grid with Tokens

Create tokens in a spatial pattern.

In [ ]:
# Create tokens in a 5x5 grid pattern (first 2 dimensions)
tokens = []
for x in range(5):
    for y in range(5):
        token_data = {
            "position": [float(x * 2), float(y * 2), 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
            "radius": 1.0,
            "weight": 1.0 + (x + y) * 0.1  # Varying weights
        }
        response = requests.post(f"{BASE_URL}/tokens", json=token_data, headers=headers)
        tokens.append(response.json())

print(f"✓ Created {len(tokens)} tokens in 5x5 pattern")

# Add tokens to grid
for token in tokens:
    response = requests.post(
        f"{BASE_URL}/grid/{grid_id}/tokens/{token['token_id']}",
        headers=headers
    )

print(f"✓ Added all tokens to grid")

## Step 4: Visualize Token Distribution

Plot tokens in 2D (first two dimensions).

In [ ]:
# Extract positions for visualization
x_coords = [t['position'][0] for t in tokens]
y_coords = [t['position'][1] for t in tokens]
weights = [t['weight'] for t in tokens]

# Plot
plt.figure(figsize=(10, 10))
scatter = plt.scatter(x_coords, y_coords, c=weights, s=200, cmap='viridis', alpha=0.6, edgecolors='black')
plt.colorbar(scatter, label='Weight')
plt.grid(True, alpha=0.3)
plt.xlabel('Dimension 0')
plt.ylabel('Dimension 1')
plt.title('Token Distribution (5x5 Grid)')
plt.axis('equal')
plt.show()

print(f"Total tokens: {len(tokens)}")

## Step 5: Find Neighbors by Radius

Query tokens within a specific distance from a point.

In [ ]:
# Find neighbors around center token (position [4.0, 4.0, ...])
center_token = [t for t in tokens if t['position'][0] == 4.0 and t['position'][1] == 4.0][0]
center_id = center_token['token_id']

# Query with radius 3.0
params = {"radius": 3.0}
response = requests.get(
    f"{BASE_URL}/grid/{grid_id}/neighbors/{center_id}",
    params=params,
    headers=headers
)

neighbors = response.json()
print(f"✓ Found {len(neighbors)} neighbors within radius 3.0")
print(f"\nCenter token: ID={center_id}, position={center_token['position'][:2]}")
print(f"\nNeighbors:")
for neighbor in neighbors[:5]:  # Show first 5
    print(f"  ID: {neighbor['token_id']}, position: {neighbor['position'][:2]}, distance: {neighbor.get('distance', 'N/A')}")

## Step 6: Visualize Neighbor Query

In [ ]:
# Plot with neighbors highlighted
plt.figure(figsize=(10, 10))

# All tokens
plt.scatter(x_coords, y_coords, c='lightgray', s=200, alpha=0.5, edgecolors='black', label='Other tokens')

# Center token
plt.scatter([center_token['position'][0]], [center_token['position'][1]], 
           c='red', s=300, marker='*', edgecolors='black', label='Center', zorder=3)

# Neighbors
neighbor_x = [t['position'][0] for t in neighbors if t['token_id'] != center_id]
neighbor_y = [t['position'][1] for t in neighbors if t['token_id'] != center_id]
plt.scatter(neighbor_x, neighbor_y, c='green', s=250, alpha=0.7, edgecolors='black', label='Neighbors', zorder=2)

# Draw search radius
circle = plt.Circle((center_token['position'][0], center_token['position'][1]), 
                    3.0, color='red', fill=False, linestyle='--', linewidth=2, label='Search radius')
plt.gca().add_patch(circle)

plt.grid(True, alpha=0.3)
plt.xlabel('Dimension 0')
plt.ylabel('Dimension 1')
plt.title('Neighbor Query (radius=3.0)')
plt.legend()
plt.axis('equal')
plt.show()

## Step 7: Range Query (Bounding Box)

Find all tokens within a rectangular region.

In [ ]:
# Query tokens in range [2.0, 6.0] x [2.0, 6.0]
range_query = {
    "min_position": [2.0, 2.0, -1.0, -1.0, -1.0, -1.0, -1.0, -1.0],
    "max_position": [6.0, 6.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
}

response = requests.post(
    f"{BASE_URL}/grid/{grid_id}/range",
    json=range_query,
    headers=headers
)

range_tokens = response.json()
print(f"✓ Found {len(range_tokens)} tokens in range")
print(f"\nRange: x=[2.0, 6.0], y=[2.0, 6.0]")
print(f"\nTokens in range:")
for token in range_tokens[:5]:
    print(f"  ID: {token['token_id']}, position: {token['position'][:2]}")

## Step 8: Visualize Range Query

In [ ]:
# Plot with range highlighted
plt.figure(figsize=(10, 10))

# All tokens
plt.scatter(x_coords, y_coords, c='lightgray', s=200, alpha=0.5, edgecolors='black', label='Outside range')

# Tokens in range
range_x = [t['position'][0] for t in range_tokens]
range_y = [t['position'][1] for t in range_tokens]
plt.scatter(range_x, range_y, c='blue', s=250, alpha=0.7, edgecolors='black', label='In range')

# Draw bounding box
from matplotlib.patches import Rectangle
rect = Rectangle((2.0, 2.0), 4.0, 4.0, linewidth=2, edgecolor='blue', facecolor='none', linestyle='--', label='Query range')
plt.gca().add_patch(rect)

plt.grid(True, alpha=0.3)
plt.xlabel('Dimension 0')
plt.ylabel('Dimension 1')
plt.title('Range Query ([2,6] x [2,6])')
plt.legend()
plt.axis('equal')
plt.xlim(-1, 9)
plt.ylim(-1, 9)
plt.show()

## Step 9: Field Influence Calculation

Calculate cumulative field strength at a point from all tokens.

In [ ]:
# Calculate field at position [5.0, 5.0, ...]
field_query = {
    "position": [5.0, 5.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
}

response = requests.post(
    f"{BASE_URL}/grid/{grid_id}/field",
    json=field_query,
    headers=headers
)

field_data = response.json()
print(f"✓ Field influence at [5.0, 5.0]:")
pprint(field_data)

## Step 10: Density Mapping

Create a heatmap of token density.

In [ ]:
# Query density in grid
response = requests.get(
    f"{BASE_URL}/grid/{grid_id}/density",
    headers=headers
)

density_data = response.json()
print(f"✓ Density map retrieved")
pprint(density_data)

## Step 11: Performance Comparison

Compare grid-based vs. brute-force search.

In [ ]:
import time

# Grid-based search
start = time.perf_counter()
response = requests.get(
    f"{BASE_URL}/grid/{grid_id}/neighbors/{center_id}",
    params={"radius": 3.0},
    headers=headers
)
grid_time = time.perf_counter() - start

print(f"Grid-based search: {grid_time * 1000:.2f}ms")
print(f"Found: {len(response.json())} neighbors")
print(f"\n✓ Grid indexing provides O(1) cell lookup!")

## Step 12: Cleanup

In [ ]:
# Delete grid
response = requests.delete(f"{BASE_URL}/grid/{grid_id}", headers=headers)
print(f"✓ Deleted grid {grid_id}")

# Delete all tokens
for token in tokens:
    requests.delete(f"{BASE_URL}/tokens/{token['token_id']}", headers=headers)

print(f"✓ Deleted {len(tokens)} tokens")

## Summary

In this tutorial, you learned:

✅ **Grid creation** - Spatial indexing with cell-based partitioning  
✅ **Neighbor queries** - Find tokens within radius  
✅ **Range queries** - Bounding box searches  
✅ **Field influence** - Calculate spatial fields  
✅ **Density mapping** - Token distribution analysis  
✅ **Performance** - O(1) grid cell lookup  

## Next Steps

- **Tutorial 3**: CDNA Profiles - Dynamic spatial behavior
- **Intermediate**: WebSocket events for real-time updates
- **Advanced**: Performance optimization techniques

## Key Takeaways

1. **Grids enable efficient spatial queries** (O(1) cell access)
2. **Neighbor search** finds tokens within radius
3. **Range queries** support bounding box searches
4. **Field calculations** model spatial influence
5. **Cell size** affects query performance and precision

---

**Need help?** Check the [API Reference](https://your-docs-url/api/index.html)